# Maximum Finding (3-bit version)

The 3-bit version is created in order to limit the qubit usage so that it can be run on the IBM-Q machine.

## The outline of algorithm

We will use 3-qbit register to store our current maximum value called `qc`. In order to find the maximum value we need to repeatedly find one larger value (index) than `qc` and stop when we can't find any. For each iteration we will use `qs` to store our larger than `qc` value. 

0. randomly initialize `qc` and measure its value
1. initialize `qs` into super position of all states 
2. apply Grover's algorithm using 3-bit comparator as the black-box function comparing `qs` and `qc` and set the input search space on `qs`
3. if `qs` is larger than `qc` replace `qc` with `qs` value and repeat step (2)
4. if `qs` is not larger than `qc` then we conclude that `qs` is the maximum value

To start we need to import all the packages we need.

In [ ]:
# importing QISKit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, execute, register, get_backend, compile

from qiskit.tools import visualization
from qiskit.tools.visualization import circuit_drawer

# import other necessary stuff
import random

import sys, time, getpass
try:
    sys.path.append("../")
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    print('Qconfig.py not found in qiskit-tutorial directory')
    
register(qx_config['APItoken'], qx_config['url'])
available_backends({'simulator': False})

To find the maximum value we need comparator first. Since we will be using multiple qubit control-cot gate, we will make a function `n_control_not()` which we will use a lot later on.

In [ ]:
def n_control_not(circuit, q_array, target, ancillary_array, flip_array=None):
    # error checking
    if flip_array is not None and len(q_array) != len(flip_array):
        raise ValueError("q_array(len:{}) and flip_array(len:{}) must have the same length".format(len(q_array), len(flip_array)))
    if len(ancillary_array) < len(q_array) - 2:
        raise ValueError("ancillary array length is not enough ({}) for q_array ({})".format(len(ancillary_array), len(q_array)))
    if flip_array is None:
        flip_array = [1 for _ in range(len(q_array))]
    # todo handle wrong flip_array
    
    n = len(q_array)
    
    # put X-gate if flip
    for i in range(n):
        if flip_array[i] == -1:
            circuit.x(q_array[i])
        
    # special case for only 2 bits
    if n == 2:
        circuit.ccx(q_array[0], q_array[1], target)
    else:
        circuit.ccx(q_array[0], q_array[1], ancillary_array[0])
        for i in range(2, n-1):
            circuit.ccx(q_array[i], ancillary_array[i-2], ancillary_array[i-1])
        circuit.ccx(q_array[n-1], ancillary_array[n-3], target)
        for i in reversed(range(2, n-1)):
            circuit.ccx(q_array[i], ancillary_array[i-2], ancillary_array[i-1])
        circuit.ccx(q_array[0], q_array[1], ancillary_array[0])
    
    # clean up X-gate
    for i in range(n):
        if flip_array[i] == -1:
            circuit.x(q_array[i])

We already have the n_control_not gate so now, onto the comparator.

Let's test out n_control_not function we just implemented
with 2 qubits (the special case) and the 4 qubits version

In [ ]:
qc = QuantumCircuit()
qs = QuantumRegister(2)
anc = QuantumRegister(1)
target = QuantumRegister(1)
qc.add(qs)
qc.add(anc)
qc.add(target)


n_control_not(qc, qs, target[0], anc)

circuit_drawer(qc)

In [ ]:
def three_bit_comparator(qc, a, b, anc, target):
    # we will store a > b for each bit in 0, 2, 4 bit (from msb to lsb) of ancillary register
    # and for b > a for each bit in 1, 3, 5 bit
    
    # TODO: reduce the qubit needed
    
    # TODO: let's check whether anc is enough
    anc_len = len(anc)
    if anc_len < 9:
        raise ValueError("ancillary bit is not enough: anc_len is {}".format(anc_len))
    
    n = 3
    
    n_control_not(qc, [a[0], b[0]], anc[0], [anc[anc_len - 1]], [1, -1]) # a > b 1st bit
    n_control_not(qc, [a[0], b[0]], anc[1], [anc[anc_len - 1]], [-1, 1]) # a < b 1st bit
    n_control_not(qc, [a[1], b[1]], anc[2], [anc[anc_len - 1]], [1, -1]) # a > b 2nd bit
    n_control_not(qc, [a[1], b[1]], anc[3], [anc[anc_len - 1]], [-1, 1]) # a < b 2nd bit
    n_control_not(qc, [a[2], b[2]], anc[4], [anc[anc_len - 1]], [1, -1]) # a > b 3rd bit
    n_control_not(qc, [a[2], b[2]], anc[5], [anc[anc_len - 1]], [-1, 1]) # a < b 3rd bit
    
    # now to the compare step
    comp_anc = [anc[6], anc[7], anc[8]]
    qc.cx(anc[0], target)
    n_control_not(qc, [anc[0], anc[1], anc[2]], target, comp_anc, [-1, -1, 1])
    n_control_not(qc, [anc[0], anc[1], anc[2], anc[3], anc[4]], target, comp_anc, [-1, -1, -1, -1, 1])
    
    # clean up the anc               
    n_control_not(qc, [a[2], b[2]], anc[5], [anc[anc_len - 1]], [-1, 1]) # a < b 3rd bit
    n_control_not(qc, [a[2], b[2]], anc[4], [anc[anc_len - 1]], [1, -1]) # a > b 3rd bit
    n_control_not(qc, [a[1], b[1]], anc[3], [anc[anc_len - 1]], [-1, 1]) # a < b 2nd bit
    n_control_not(qc, [a[1], b[1]], anc[2], [anc[anc_len - 1]], [1, -1]) # a > b 2nd bit
    n_control_not(qc, [a[0], b[0]], anc[1], [anc[anc_len - 1]], [-1, 1]) # a < b 1st bit
    n_control_not(qc, [a[0], b[0]], anc[0], [anc[anc_len - 1]], [1, -1]) # a > b 1st bit

Let's see how the 3-qubit comparator looks like.

In [ ]:
qc = QuantumCircuit()
q = QuantumRegister(6)
anc = QuantumRegister(9)
target = QuantumRegister(1)
qc.add(q)
qc.add(anc)
qc.add(target)

three_bit_comparator(qc, [q[0], q[1], q[2]], [q[3], q[4], q[5]], anc, target[0])

circuit_drawer(qc)

The next step is use the comparator and do Grover's search to find a larger value. We'll be using Boyer et al version of Grover's algorithm which goes as follow.

1. initialize $m = 1$ and set $\lambda = 6/5$
2. choose `j` uniformly at random among the nonnegative integers smaller than `m`
3. apply j iterations of Grover's algorithm starting from initial state $\left|\Psi_0\right\rangle = \sum_i \frac{1}{\sqrt{N}}\left|i\right\rangle$
4. observe the register: let $i$ be the outcome
5. if `T[i] = x`, the problem is solved: **exit**
6. otherwise, set `m` to min($\lambda m$, $\sqrt{N}$)    
and go back to step 2.

We will need inverse around the mean function

In [ ]:
def inv_around_mean(circuit, qs, anc):
    n = len(qs)
    for i in range(n):
        circuit.h(qs[i])
    for i in range(n):
        circuit.x(qs[i])
    circuit.h(qs[n-1])
    n_control_not(circuit, [qs[i] for i in range(n-1)], qs[n-1], anc)
    circuit.h(qs[n-1])
    for i in range(n):
        circuit.x(qs[i])
    for i in range(n):
        circuit.h(qs[i])

And now for the main part, finding a larger value using Grover's algorithm.

In [ ]:
def int_to_qubit(n, circuit, qs):
    if n >= 8:
        raise ValueError('this function does not support n larger than 7, input n = {}'.format(n))
    if len(qs) < 3:
        raise ValueError('input registers is less than 3 qubits')
    bstr = '{0:03b}'.format(n)
    for i in range(3):
        if bstr[-1 + (-1 * i)]:
            circuit.x(qs[i])

def init_grover(circuit, f_input, f_output):
    for j in range(len(f_input)):
        circuit.h(f_input[j])
    circuit.x(f_output)
    circuit.h(f_output)

def grover_find_larger(current_max, device='local_qasm_simulator'):
    """
    return value:
        max_value: highest value found
        iterations: number of iterations run
        function_evaluation_counts: number of function evaluated
    """
    m = 1
    lamb = 6/5
    j = random.randrange(m)
    
    f_eval_count = 0
    iterations = 1
    
    # we do this until we can't find any larger
    while iterations <= 10:
        # init quantum circuit
        qc = QuantumCircuit()
        a = QuantumRegister(3)
        b = QuantumRegister(3)
        anc = QuantumRegister(9)
        target = QuantumRegister(1)
        cs = ClassicalRegister(3)
        qc.add(a)
        qc.add(b)
        qc.add(anc)
        qc.add(target)
        qc.add(cs)

        # set current max to b
        int_to_qubit(current_max, qc, b)

        init_grover(qc, a, target)
    
        for t in range(m):
            # Apply m iterations
            three_bit_comparator(qc, a, b, anc, target[0])
            inv_around_mean(qc, a, anc)
            f_eval_count += 1
        
        qc.measure(a, cs)
  
        # Execute circuit
        job = execute([qc], backend=device, shots=1)
        
        result = job.result()

        counts = result.get_counts(qc)
#         print('iteration #{}: results: {}, func_eval_count = {}'.format(iterations, counts, f_eval_count))
#         print('iteration #{}: results: {}, func_eval_count = {}'.format(iterations, int(next(iter(counts)), 2), f_eval_count))
        iterations += 1
        m = int(min(lamb * m, 3))
        new_max = int(next(iter(counts)), 2)
        if new_max > current_max:
            current_max = new_max
            break
    return (current_max, iterations, f_eval_count)

We already implemented all the components we need in order to run maximum finding algorithm.

Let's start with the simplest case, the case where every index of search space corresponds to the index value itself.

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

    # maximum finding scenario 1: f[x] = x

    n = 0
    iterations = 0
    f_count = 0
    loop = 0

    while True:
        nn, nit, nfc = grover_find_larger(n)
        iterations += nit
        f_count += nfc
        loop += 1

        print('loop #{}: max = {}, called with {}, result = {}, f_count = {}'.format(loop, max(n, nn), n, nn, f_count))
        if n >= nn:
            break
        else:
            n = nn
        
print('............finished..............')


### Now let's test the same one with real devices.

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

    # maximum finding scenario 1: f[x] = x

    n = 0
    iterations = 0
    f_count = 0
    loop = 0

    while True:
        nn, nit, nfc = grover_find_larger(n, 'ibmqx5')
        iterations += nit
        f_count += nfc
        loop += 1

        print('loop #{}: max = {}, called with {}, result = {}, f_count = {}'.format(loop, max(n, nn), n, nn, f_count))
        if n >= nn:
            break
        else:
            n = nn
        
print('............finished..............')